Import Libraries

In [3]:
import pandas as pd
import math


Import csv

In [4]:
df = pd.read_csv('modelinput.csv')

Helper Function to calculate distances

In [5]:
def calculate_distance(x1, y1, x2, y2):

    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

In [14]:
def get_angle(arm_x,arm_y, base_x, base_y):

    # Translate ball position to base line / co-ordinates
    std_ball = [arm_x - base_x, arm_y - base_y]

    ang_rad = math.tan(std_ball[1]/std_ball[0])

    return ang_rad * (180/math.pi)


In [120]:
def pos_angle(num):
    if abs(num) > 360:
        full_rot = abs(num)//360
        if num > 0: 
            num-= int(full_rot)*360
        else:
            num += int((full_rot))*360

    if num < 0:
        num += 360

    return num




Create New Data Frame and fill in rows

In [121]:
# Initialize variables and DataFrames
final_df = pd.DataFrame()
new_row1 = {}
new_row2 = {}
new_row3 = {}
begun = False
play_start = False
count = 0

# Iterate through rows in df
for index, row in df.iterrows():
    play_id = row['play_id']
    if row['event_code'] == 1:
        play_start = True
    
    if play_start:
        count +=1
        if not begun:
            if row['event_code'] == 4 and isinstance(eval(row['ball_pos']), tuple):

                begun = True
                start_time = row['timestamp']

                ball_start_x = eval(row['ball_pos'])[0]
                ball_start_y = eval(row['ball_pos'])[1]

                # Get player initial positions
                starting_places = eval(row['player_positions'])

                ''' 
                Create baseline here
                '''


                
        elif row['event_code'] != 4000:
            ball_pos = eval(row['ball_pos'])

            if isinstance(ball_pos, tuple) and isinstance(starting_places, list) and len(starting_places)==3:

                '''
                1.Create a line from home plate to initial player position 
                    - This will be 0 degrees
                2. Find angle of final ball position relative to base line
                '''

                # Get ball final positions
                ball_x, ball_y = ball_pos[0], ball_pos[1]

                # Get angle offsets to make sure each fielder has standard angle meaning

                offset_angle_1 = 90 - get_angle(starting_places[0][7][0], starting_places[0][7][1], ball_start_x, ball_start_y)
                offset_angle_2 = 90 - get_angle(starting_places[1][8][0], starting_places[1][8][1], ball_start_x, ball_start_y)
                offset_angle_3 = 90 - get_angle(starting_places[2][9][0], starting_places[2][9][1], ball_start_x, ball_start_y)



                ang_rellative_1 = pos_angle(get_angle(ball_x, ball_y, starting_places[0][7][0], starting_places[0][7][1]) + offset_angle_1)
                ang_rellative_2 = pos_angle(get_angle(ball_x, ball_y, starting_places[1][8][0], starting_places[1][8][1]) + offset_angle_2)
                ang_rellative_3 = pos_angle(get_angle(ball_x, ball_y, starting_places[2][9][0], starting_places[2][9][1]) + offset_angle_3)

                



                distance_1 = calculate_distance(ball_x, ball_y, starting_places[0][7][0], starting_places[0][7][1])
                distance_2 = calculate_distance(ball_x, ball_y, starting_places[1][8][0], starting_places[1][8][1])
                distance_3 = calculate_distance(ball_x, ball_y, starting_places[2][9][0], starting_places[2][9][1])




                
                if row['event_code'] == 2:
                    catch_positions = eval(row['catch_position'])

                    new_row1 = {'distance': distance_1, 'time': row['timestamp'] - start_time, 'angle': ang_rellative_1, 'catch': catch_positions.get(7, 0)}
                    new_row2 = {'distance': distance_2, 'time': row['timestamp'] - start_time,'angle': ang_rellative_2, 'catch': catch_positions.get(8, 0)}
                    new_row3 = {'distance': distance_3, 'time': row['timestamp'] - start_time,'angle': ang_rellative_3, 'catch': catch_positions.get(9, 0)}
                else:
                    new_row1 = {'distance': distance_1, 'time': row['timestamp'] - start_time,'angle': ang_rellative_1, 'catch': 0}
                    new_row2 = {'distance': distance_2, 'time': row['timestamp'] - start_time,'angle': ang_rellative_2, 'catch': 0}
                    new_row3 = {'distance': distance_3, 'time': row['timestamp'] - start_time, 'angle': ang_rellative_3,'catch': 0}

                # Convert dictionaries to DataFrames
                df_new_row_1 = pd.DataFrame(new_row1, index=[0])
                df_new_row_2 = pd.DataFrame(new_row2, index=[0])
                df_new_row_3 = pd.DataFrame(new_row3, index=[0])

                # Concatenate DataFrames to final_df
                final_df = pd.concat([final_df, df_new_row_1, df_new_row_2, df_new_row_3], ignore_index=True)

                # Reset new_row dictionaries
                new_row1 = {}
                new_row2 = {}
                new_row3 = {}

            begun = False
            play_start = False

Export as csv

In [131]:
print (final_df)

         distance  time       angle  catch
0      133.321061  5500  351.558070      0
1       23.774403  5500  115.469087      1
2      163.004893  5500  188.731762      0
3       59.623193  5150   45.448349      1
4      162.019367  5150  142.124675      0
...           ...   ...         ...    ...
10525  188.289394  6300   23.187083      0
10526   69.157659  6300  351.024277      1
10527  199.475985  5800   25.474118      0
10528   50.665847  5800  132.204260      1
10529   78.973925  5800  151.834119      0

[10530 rows x 4 columns]


In [133]:
final_df.to_csv('modelData_1.csv', index=False)

<strong>Notes:</strong>
To remove angles from dataframe and save to csv for model input run the following cell

In [130]:
#without angle
df = final_df.drop(columns=['angle'])
df.to_csv('modelData_1.csv', index=False)